## 1. 파이썬과 MySQL 활용

+ 파이썬과 MySQL을 연결해주는 모듈 : mysqlclient, PyMySQL
    - pip install mysqlclient
    - pip install PyMySQL

In [11]:
##### mysqlclient 모듈을 이용한 연결 테스트 

import MySQLdb

conn = MySQLdb.connect(host="127.0.0.1", user="root", passwd="1111", database="testdb")
#conn = MySQLdb.connect(host="netsong7.synology.me", user="user2", 
#                       password="Mysqldb12345!", database="user2db", port=3307)
print(conn)
conn.close()

<_mysql.connection open to '127.0.0.1' at 00000197657E73E0>


In [10]:
##### PyMySQL 모듈을 이용한 연결 테스트 

import pymysql

conn = pymysql.connect(host="127.0.0.1", user="root", passwd="1111", database="testdb")
print(conn)
conn.close()

In [12]:
config = {"host":"127.0.0.1", "user":"root", "passwd":"1111", "database":"testdb"}

In [13]:
conn = pymysql.connect(**config)
print(conn)
conn.close()

### (1) SELECT

In [17]:
conn = MySQLdb.connect(**config)

sql = "select * from emp limit 5"

cur = conn.cursor()
cur.execute(sql)

# print(cur)

#for data in cur:
#    print(data)

#for a, b, c, d, e, f, g, h in cur:
#    print(a, b, c, d, e, f, g, h)

result = []
for data in cur:
    result.append(data)
    
conn.close()

print(result)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)]


### (2) INSERT

In [23]:
conn = MySQLdb.connect(**config)

sql = """create table if not exists tbltest(
    empno    int,
    ename    varchar(10),
    sal      int,
    job      varchar(10)
)"""

cur = conn.cursor()
cur.execute(sql)

sql = "insert into tbltest(empno, ename, sal) values(1, 'scott', 5000)"
cur.execute(sql)
conn.commit()

conn.close()

In [28]:
empno = 2
ename = "miller"
sal = 1300

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "insert into tbltest(empno, ename, sal) values(" \
    + str(empno) + ", '" + ename + "', " + str(sal)+ ")"
    
#print(sql)
cur.execute(sql)
conn.commit()

conn.close()

In [29]:
empno = 3
ename = "tom"
sal = 2000

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
sql_data = (str(empno), ename, str(sal))

cur.execute(sql, sql_data)
conn.commit()

conn.close()

In [30]:
empno = 4
ename = "jerry"
sal = 2000

try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    sql = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
    sql_data = (str(empno), ename, str(sal))

    cur.execute(sql, sql_data)
    conn.commit()
except Exception as e:
    print("error : ", e)
finally:
    conn.close()

In [31]:
##### 데이터 여러 개 입력하기

conn = MySQLdb.connect(**config)
cur = conn.cursor()

for i in range(10):
    sql = "insert into tbltest(empno, ename, sal) values(5, 'jane', 10000)"
    cur.execute(sql)
    conn.commit()
    
conn.close()

### (3) Cursor

#### 1) fetchall(), fetchmany(), fetchone()

+ default는 fetchall()

In [35]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("select * from tbltest")
    
    #for i in cur.fetchall():
    #    print(i)
    
    #for i in cur.fetchmany(5):
    #    print(i)
    
    print(cur.fetchone())
        
finally:
    conn.close()

(1, 'scott', 5000, None)


#### 2) DictCursor

+ 필드명을 인덱스 대신 키로 사용

In [39]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor(MySQLdb.cursors.DictCursor)
    
    cur.execute("select * from tbltest")
    
    for i in cur.fetchall():
        # print(i[0], i[1], i[2])
        print(i["empno"], i["ename"], i["sal"])
    
finally:
    conn.close()

1 scott 5000
1 scott 5000
2 miller 1300
3 tom 2000
4 jerry 2000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000


## 2. 응용 예제

### (1) CSV파일을 DB에 저장

In [ ]:
"""
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);

"""

In [56]:
import csv

file = csv.reader(open("data/input.csv"))
#print(file)

#print(next(file))
#print(next(file))
#print(next(file))
#print(next(file))
#print(next(file))

next(file)

try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("delete from suppliers")
    conn.commit()
    
    for row in file:
        # print(row)
        cur.execute("insert into suppliers values(%s, %s, %s, %s, %s)", row)
    
    conn.commit()
    
finally:
    conn.close()

### (2) DB에서 가져온 데이터를 파일로 저장

In [61]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("select * from suppliers where cost > 500000")
    
    #for i in cur:
    #    print(i)
    
    file = csv.writer(open("data/output.csv", "w"), delimiter=",")
    
    header = ["Supplier Name","Invoice Number","Part Number","Cost","Purchase Date"]
    file.writerow(header)
    
    for row in cur:
        file.writerow(row)
    
finally:
    conn.close()